# Geostrophic Balance

Here we study the surface currents modeled by the 2D incompressible Navier-Stokes equations on a rotating sphere:

$$ u_{t} + (\mathbf{u} \cdot \nabla)u + fv = -\frac{1}{\rho} p_{x} $$
$$ v_{t} + (\mathbf{u} \cdot \nabla)v - fu = -\frac{1}{\rho} p_{y} $$

where $\rho$ is the density, x and y are the zonal and meridional coordinates respectively. $f$, the Coriolis parameter is given in terms of the Earth's rotation rate $\Omega$ and the latitude $\phi$.

For this case study, the data is obtainable from this [URL](https://data.hycom.org/datasets/GOMu0.04/expt_50.1/data/netcdf/1993/) directly. The fields used are the first and second fields of the hindcasts (see below).

This code was written based on the paper and the supplementary information provided.

In [37]:
import xarray as xr
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture

## 1. Load the data


In [21]:
# Read in the data from the nc files at 2 times to get time derivatives
path1 = "../Data/hycom_gomu_501_1993010100_t000.nc"
path2 = "../Data/hycom_gomu_501_1993010100_t003.nc"
data_t1 = xr.open_dataset(path1, decode_times=False)
data_t2 = xr.open_dataset(path2, decode_times=False)

In [22]:
# Visualise what the data looks like
data_t1

<xarray.Dataset>
Dimensions:     (time: 1, depth: 40, lat: 346, lon: 541)
Coordinates:
  * time        (time) float64 -6.134e+04
  * depth       (depth) float64 0.0 2.0 4.0 6.0 ... 2.5e+03 3e+03 4e+03 5e+03
  * lat         (lat) float64 18.12 18.16 18.2 18.24 ... 31.8 31.84 31.88 31.92
  * lon         (lon) float64 -98.0 -97.96 -97.92 -97.88 ... -76.48 -76.44 -76.4
Data variables:
    tau         (time) float64 ...
    water_u     (time, depth, lat, lon) float32 ...
    water_v     (time, depth, lat, lon) float32 ...
    water_temp  (time, depth, lat, lon) float32 ...
    salinity    (time, depth, lat, lon) float32 ...
    surf_el     (time, lat, lon) float32 ...
Attributes:
    classification_level:      UNCLASSIFIED
    distribution_statement:    Approved for public release. Distribution unli...
    downgrade_date:            not applicable
    classification_authority:  not applicable
    institution:               Naval Oceanographic Office
    source:                    HYCOM archive file
    history:                   archv2ncdf3z
    field_type:                instantaneous
    Conventions:               CF-1.6 NAVO_netcdf_v1.1

In [23]:
# Get the necessary variables
lat = data_t1["lat"].values
lon = data_t1["lon"].values

u = np.array([data_t1["water_u"].values, data_t2["water_u"].values])
u = u[:, 0, 0, :, :]

v = np.array([data_t1["water_v"].values, data_t2["water_v"].values])
v = v[:, 0, 0, :, :]

# Set some domain characteristics
nx = len(lon)
ny = len(lat)
omega = 7.29e-5  # rad/s
f = 2 * omega * np.sin(lat * np.pi / 180)
rho = 1030  # kg/m^3, assuming constant density

In [24]:
u.shape

(2, 346, 541)

In [25]:
# Set the time and space step sizes
R = 6.37e6  # m, Earth radius
dt = 3 * 3600  # 3 hours
# Because dx is variable due to it varying with latitude,
# we'll use np.gradients and the position of x to get the x derivative of variables
x = (1 / 25) * (1 / 360) * R * np.cos(lat * np.pi / 180) * np.pi / 180
dy = 110000 * (1 / 25) * (1 / 360)  # Using a 1 degree of lat = 110 km assumption

## 2. Getting the derivatives

From the studied equation, we need x and y derivatives of u, v, and p, and the time derivative of u and v.

For this, we'll use finite difference, following the alternative methods written for the Boundary Layer case (see Boundary_Layer.ipynb)

In [26]:
def get_derivatives_numpy_geo(nx, ny, x, dy, u, v):
    """Get the derivatives for the 2D domain

    Parameters:
    -----------
    nx: int
        Number of points in the x-direction
    ny: int
        Number of points in the y-direction
    dx: float
        Step in the x-direction
    y: numpy array
        Array of y coordinates
    u: numpy array
        Array of u velocities
    v: numpy array
        Array of v velocities


    Returns:
    --------
    u_x: numpy array
        x-derivative of u
    u_y: numpy array
        y-derivative of u
    v_x: numpy array
        x-derivative of v
    v_y: numpy array
        y-derivative of v
    """

    u_x = np.zeros((ny, nx))
    u_y = np.zeros((ny, nx))
    v_x = np.zeros((ny, nx))
    v_y = np.zeros((ny, nx))

    u_x = np.gradient(u, x, axis=0)
    v_x = np.gradient(v, x, axis=0)

    u_y = np.gradient(u, dy, axis=1)
    v_y = np.gradient(v, dy, axis=1)

    return u_x, u_y, v_x, v_y

In [27]:
# Get the derivatives
u_x, u_y, v_x, v_y = get_derivatives_numpy_geo(nx, ny, x, dy, u[0], v[0])

In [34]:
# Get the time derivatives
u_t = (u[1] - u[0]) / dt
v_t = (v[1] - v[0]) / dt

# Define the terms
u_grad_u = u[0] * u_x + v[0] * u_y
u_grad_v = u[0] * v_x + v[0] * v_y

# Get the Coriolis term
f = f.reshape(ny, 1)
f_u = f * u[0]
f_v = f * v[0]

# Get the pressure gradient term
p_x = -rho * (u_t + u_grad_u + f_v)
p_y = -rho * (v_t + u_grad_v - f_u)

In [35]:
# Define the features

# Flatten the terms
u_t = u_t.flatten()
v_t = v_t.flatten()
u_grad_u = u_grad_u.flatten()
u_grad_v = u_grad_v.flatten()
f_u = f_u.flatten()
f_v = f_v.flatten()
p_x = p_x.flatten()
p_y = p_y.flatten()

# print the shapes
print(
    u_t.shape,
    v_t.shape,
    u_grad_u.shape,
    u_grad_v.shape,
    f_u.shape,
    f_v.shape,
    p_x.shape,
    p_y.shape,
)

(187186,) (187186,) (187186,) (187186,) (187186,) (187186,) (187186,) (187186,)


In [36]:
# Stack the features
features_mer = pd.DataFrame(
    {
        "u_t": u_t,
        "u_grad_u": u_grad_u,
        "f_v": f_v,
        "p_x": p_x,
    }
)

features_zon = pd.DataFrame(
    {
        "v_t": v_t,
        "u_grad_v": u_grad_v,
        "f_u": f_u,
        "p_y": p_y,
    }
)

features_mer = features_mer.to_numpy()
features_zon = features_zon.to_numpy()

nfeatures = 4

## 3. Gaussian Mixture Model Clustering

In [39]:
seed = 75016
np.random.seed(seed)

model = GaussianMixture(n_components=6, random_state=seed)
model.fit(features_mer)

ValueError: Input X contains NaN.
GaussianMixture does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values